In [1]:
import sys
sys.path.insert(0, '/Users/shreemit/Developer/movie-recs')
import numpy as np
from tqdm.notebook import tqdm

In [2]:
import pandas as pd
from filters.colab_filter import CollaborativeMovieRecommender
from filters.content_based import ContentBasedRecommender
from filters.recommender import Recommender


/Users/shreemit/miniforge3/envs/leaseGPT/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
test_data = pd.read_csv('/Users/shreemit/Developer/movie-recs/test_data.csv')

In [4]:
recommender = Recommender(ratings_path='/Users/shreemit/Developer/movie-recs/train_data.csv',
                            movies_path='/Users/shreemit/Developer/movie-recs/merge_movies.csv',
                            movie_features_path='/Users/shreemit/Developer/movie-recs/movie_features.pkl')
recommended_movies = recommender.recommend_movies(user_id=2, method='content_based')
print(recommended_movies)

[7282, 7408, 24, 7316, 7409]


In [28]:
# looping through the test data to get the recommended movies for each user
results_colab = []
for user_id in tqdm(test_data['userId'].unique()):
    recommended_movies = recommender.recommend_movies(method='colab_filter',
                                                      user_id=user_id)
    results_colab.append({'userId': user_id, 'movieId': recommended_movies})


  0%|          | 0/610 [00:00<?, ?it/s]

In [31]:
results_colab

[{'userId': 1, 'movieId': [344, 1793, 5989, 590, 5796]},
 {'userId': 2, 'movieId': [131724, 89774, 2959, 71579, 114060]},
 {'userId': 3, 'movieId': [98623, 5746, 7899, 163937, 5181]},
 {'userId': 4, 'movieId': [930, 106, 2203, 2066, 4260]},
 {'userId': 5, 'movieId': [6062, 2571, 158, 6041, 1196]},
 {'userId': 6, 'movieId': [592, 93, 364, 588, 313]},
 {'userId': 7, 'movieId': [2126, 86644, 47, 3513, 60514]},
 {'userId': 8, 'movieId': [21, 590, 588, 440, 480]},
 {'userId': 9, 'movieId': [3735, 1291, 6006, 5954, 2012]},
 {'userId': 10, 'movieId': [8869, 102445, 72330, 113394, 94864]},
 {'userId': 11, 'movieId': [848, 4993, 47, 4688, 2571]},
 {'userId': 12, 'movieId': [256, 7033, 416, 325, 49957]},
 {'userId': 13, 'movieId': [6365, 1947, 5582, 3578, 8910]},
 {'userId': 14, 'movieId': [156, 380, 533, 2073, 236]},
 {'userId': 15, 'movieId': [5989, 1958, 260, 1291, 3925]},
 {'userId': 16, 'movieId': [6377, 102445, 6539, 8874, 68157]},
 {'userId': 17, 'movieId': [7153, 8195, 260, 1196, 4289]},

In [32]:
results_content = []
for user_id in tqdm(test_data['userId'].unique()):
    recommended_movies = recommender.recommend_movies(method='content_based',
                                                      user_id=user_id)
    results_content.append({'userId': user_id, 'movieId': recommended_movies})


  0%|          | 0/610 [00:00<?, ?it/s]

In [71]:
# preparing dict for test data {userId: [movieId]}
test_dict = {}
# subset of movies with ratings >= 3
user_liked_movies = test_data[test_data['rating'] >= 3]
for user_id in user_liked_movies['userId'].unique():
    test_dict[user_id] = user_liked_movies[user_liked_movies['userId'] == user_id]['movieId'].values


In [72]:
user_liked_movies.head()

,userId,movieId,rating,title,genres,year,imdbId,tmdbId
0,1,3578,5.0,Gladiator,Action|Adventure|Drama,2000.0,172495.0,98.0
1,1,1127,4.0,"Abyss, The",Action|Adventure|Sci-Fi|Thriller,1989.0,96754.0,2756.0
2,1,157,5.0,Canadian Bacon,Comedy|War,1995.0,109370.0,1775.0
4,1,260,5.0,Star Wars: Episode IV - A New Hope,Action|Adventure|Sci-Fi,1977.0,76759.0,11.0
5,1,3053,5.0,"Messenger: The Story of Joan of Arc, The",Drama|War,1999.0,151137.0,10047.0


In [73]:
len(test_dict)

609

In [74]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [75]:
def evaluate_recommendations(results_content, test_data):
    binary_recommendations = np.array([])
    for result in results_content:
        if result['userId'] not in test_dict:
            print(f"Error: User ID {result['userId']} not found in test data.")
            continue
        test_data = set(test_dict[result['userId']])
        # binary_recommendation = [int(movie in test_data) for movie in result['movieId']]
        for movie in result['movieId']:
                # print(result['userId'])
            binary_recommendation = [int(movie in test_data)]
            if movie in test_data:
                break
        binary_recommendations = np.append(binary_recommendations,binary_recommendation)
        
    print(np.count_nonzero(binary_recommendations))
    # Calculate precision, recall, and accuracy
    true_positives = np.sum(binary_recommendations)
    predicted_positives = len(binary_recommendations)
    actual_positives = len(test_data)

    precision = true_positives / predicted_positives
    recall = true_positives / actual_positives
    accuracy = true_positives / len(results_content)

    # Display the results
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'Accuracy: {accuracy}')


    return binary_recommendations

In [76]:
res = evaluate_recommendations(results_content, test_dict)

Error: User ID 442 not found in test data.
51
Precision: 0.08374384236453201
Recall: 0.0893169877408056
Accuracy: 0.08360655737704918


In [77]:
res = evaluate_recommendations(results_colab, test_dict)

Error: User ID 442 not found in test data.
238
Precision: 0.39080459770114945
Recall: 0.4168126094570928
Accuracy: 0.3901639344262295


In [78]:
np.count_nonzero(res)

238

In [79]:
(test_data['userId'].nunique()), len(results_content)

(610, 610)

In [80]:
results_colab

[{'userId': 1, 'movieId': [344, 1793, 5989, 590, 5796]},
 {'userId': 2, 'movieId': [131724, 89774, 2959, 71579, 114060]},
 {'userId': 3, 'movieId': [98623, 5746, 7899, 163937, 5181]},
 {'userId': 4, 'movieId': [930, 106, 2203, 2066, 4260]},
 {'userId': 5, 'movieId': [6062, 2571, 158, 6041, 1196]},
 {'userId': 6, 'movieId': [592, 93, 364, 588, 313]},
 {'userId': 7, 'movieId': [2126, 86644, 47, 3513, 60514]},
 {'userId': 8, 'movieId': [21, 590, 588, 440, 480]},
 {'userId': 9, 'movieId': [3735, 1291, 6006, 5954, 2012]},
 {'userId': 10, 'movieId': [8869, 102445, 72330, 113394, 94864]},
 {'userId': 11, 'movieId': [848, 4993, 47, 4688, 2571]},
 {'userId': 12, 'movieId': [256, 7033, 416, 325, 49957]},
 {'userId': 13, 'movieId': [6365, 1947, 5582, 3578, 8910]},
 {'userId': 14, 'movieId': [156, 380, 533, 2073, 236]},
 {'userId': 15, 'movieId': [5989, 1958, 260, 1291, 3925]},
 {'userId': 16, 'movieId': [6377, 102445, 6539, 8874, 68157]},
 {'userId': 17, 'movieId': [7153, 8195, 260, 1196, 4289]},

In [81]:
for result in results_colab:
    test_set = set(test_dict[result['userId']])
    for movie in result['movieId']:
        if movie in test_set:
            print(result['userId'])
        

6
6
8
8
13
17
18
20
22
23
24
25
26
27
30
30
37
37
39
40
40
44
44
45
50
52
54
54
56
57
63
64
64
68
71
74
76
77
80
81
82
86
88
91
91
94
94
95
97
102
104
109
111
112
112
113
114
115
116
117
117
122
124
126
128
129
130
132
132
134
135
135
136
138
139
142
144
144
150
152
159
160
161
161
164
166
166
167
171
171
174
181
188
189
189
190
191
191
195
199
202
204
205
209
215
219
220
221
223
230
232
233
234
239
239
239
242
244
247
251
252
258
261
263
263
267
269
270
274
276
282
282
284
284
285
288
288
290
295
296
296
300
301
301
303
307
308
310
312
312
321
323
328
328
330
330
332
334
334
335
339
340
343
343
344
347
348
348
349
350
351
352
352
354
357
366
366
368
372
373
374
376
378
386
393
394
396
399
399
400
400
402
402
403
404
407
409
411
413
415
417
420
421
425
426
426
426
432
432
434
437
438
439
439
440
440
441


KeyError: 442

In [82]:
# manually checking the results
for result in results_content[0]['movieId']:
    movie = movies[movies['movieId'] == result]
    print(result)
    print(str(movie['title']))

NameError: name 'movies' is not defined

In [ ]:
results_content[0]['movieId']